In [1]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix, classification_report

from flows.autoregressive import MaskedAutoregressiveFlow
from utils import load_flow

FLOW_PATH = 'logs/MAF/version_0/'

In [2]:
flow = load_flow(FLOW_PATH).cuda()

In [3]:
def summarise(z, y):
    z = torch.tensor(z.values, dtype=torch.float).cuda()

    y0 = torch.zeros(z.shape[0], 1).cuda()
    y1 = torch.ones(z.shape[0], 1).cuda()
    
    with torch.no_grad():
        res0 = flow.log_prob(z, y0).detach().cpu().numpy()
        res1 = flow.log_prob(z, y1).detach().cpu().numpy()
        
    df = pd.DataFrame({'res0': res0, 'res1': res1, 'y': y.values[:, 0]})
    df['y_hat'] = (df['res0'] < df['res1']).astype(int)
    
    print(confusion_matrix(df['y'], df['y_hat']))
    print(classification_report(df['y'], df['y_hat']))

In [4]:
z_train = pd.read_csv('data/z_train.csv')
y_train = pd.read_csv('data/y_train_gender.csv')

z_test = pd.read_csv('data/z_test.csv')
y_test = pd.read_csv('data/y_test_gender.csv')

In [5]:
summarise(z_train, y_train)

[[74261 20248]
 [13435 54826]]
              precision    recall  f1-score   support

           0       0.85      0.79      0.82     94509
           1       0.73      0.80      0.77     68261

    accuracy                           0.79    162770
   macro avg       0.79      0.79      0.79    162770
weighted avg       0.80      0.79      0.79    162770



In [6]:
summarise(z_test, y_test)

[[9549 2698]
 [1569 6146]]
              precision    recall  f1-score   support

           0       0.86      0.78      0.82     12247
           1       0.69      0.80      0.74      7715

    accuracy                           0.79     19962
   macro avg       0.78      0.79      0.78     19962
weighted avg       0.80      0.79      0.79     19962

